In [1]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import regularizers
import os
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.optimizers import Adam
import time
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
sv_df = pd.read_csv("/content/drive/MyDrive/Sound_Vector.csv")

In [20]:
from sklearn.utils import shuffle
sv_df_ = shuffle(sv_df)
sv_df_[:10]

,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,0.1
44096,-23.988155,-25.665876,-26.145317,-25.937980,-27.315960,-27.954943,-15.654886,-10.105742,-10.636259,-12.053652,...,-15.864976,-18.219454,-18.853554,-19.163311,-22.993359,-37.077591,-43.038731,-44.020927,-44.162399,female_neutral
31903,-5.785833,-7.357610,-12.952780,-15.747028,-16.366428,-10.230941,-7.612992,-9.266970,-10.295159,-9.419586,...,-39.731682,-37.936409,-37.462898,-40.013371,-42.548874,-43.666855,-43.618755,-42.695164,-44.097073,male_nervous
41647,-5.014238,-6.653152,-12.651302,-13.268180,-15.012000,-16.404146,-16.280058,-16.378302,-16.530722,-15.247422,...,-8.518603,-9.733719,-9.832970,-11.175131,-14.734213,-18.780920,-20.606730,-24.284018,-27.238209,female_neutral
18187,-6.916595,-9.880234,-20.767700,-25.634756,-29.479940,-32.286040,-34.792107,-35.950527,-37.936240,-31.170895,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,female_angry
23923,-30.003704,-26.823978,-27.506962,-26.347353,-17.376194,-13.719575,-15.901433,-18.925648,-19.350882,-20.420650,...,-23.895021,-20.732883,-18.193510,-20.181396,-24.455566,-25.329182,-26.300680,-24.546568,-19.772444,male_angry
10270,-29.641792,-30.370304,-33.561550,-34.319260,-36.172707,-36.822754,-36.372856,-34.686913,-26.776943,-21.573961,...,-21.179918,-19.820404,-20.586138,-20.858919,-19.056467,-18.678850,-21.067423,-17.524742,-10.793470,male_sad
32248,-1.199229,-1.506416,-4.963626,-4.673806,-5.564949,-7.143599,-8.239462,-9.325773,-9.702705,-9.447771,...,-4.427224,-4.786947,-5.415229,-6.688566,-9.122552,-7.060706,-5.566640,-3.024532,-1.595626,female_surprise
44791,-4.540023,-2.871056,-4.525616,-4.090779,-4.043938,-5.798038,-9.169870,-8.325746,-8.013352,-6.414149,...,-15.055876,-16.454966,-13.566071,-11.068525,-10.432170,-7.947036,-7.694471,-5.215118,-3.929293,male_neutral
38479,0.374446,1.671105,0.339486,0.148823,0.200620,0.540976,-0.108713,-1.239041,-4.244438,-4.580957,...,-8.449316,-8.780019,-9.258269,-8.867668,-9.058729,-5.706816,-6.110885,-5.214762,-3.223830,male_surprise
22634,-7.492559,-11.427984,-27.324318,-28.517841,-23.497784,-20.115770,-18.432955,-19.506052,-23.501616,-25.015125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,male_angry


In [21]:
sv_df_=sv_df_.fillna(0)

In [22]:
X = sv_df_.drop(sv_df_.iloc[:, [-1]], axis = 1)
y = sv_df_.iloc[:, [-1]]

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [25]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
9947,-11.382583,-15.128574,-24.922537,-21.549946,-20.067293,-20.861328,-21.765790,-26.746714,-27.626810,-25.785420,...,-34.291904,-36.811596,-42.471306,-43.718365,-45.077030,-46.098000,-46.356594,-46.113281,-46.349102,-47.511501
46637,0.931464,0.970379,-5.559112,-6.779528,-10.593250,-12.890318,-12.541811,-16.030360,-13.847661,-7.535757,...,-11.413024,-12.232421,-12.419788,-12.433274,-10.995432,-9.904491,-11.010721,-11.333757,-7.615479,-5.000741
23205,-12.134127,-16.482393,-26.854755,-26.982594,-27.274527,-26.289059,-22.449436,-21.799894,-22.402107,-20.494734,...,-33.990231,-31.959801,-25.344460,-23.720310,-26.659172,-35.819973,-32.929905,-23.233517,-16.511450,-8.874736
17272,-0.439662,-3.611537,-16.695604,-16.408787,-17.803144,-16.148111,-15.721159,-13.945725,-12.092894,-11.833383,...,-14.465506,-15.853846,-16.769646,-17.862759,-20.426332,-19.727631,-17.425632,-18.089945,-16.278902,-10.258831
46998,-19.612795,-15.062821,-11.878865,-12.968856,-14.803028,-15.136979,-13.586335,-13.590147,-17.157839,-9.130546,...,-24.317196,-27.555998,-28.536947,-30.982227,-33.029076,-34.563873,-34.742912,-36.115940,-36.857719,-39.378170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31681,-4.017278,-6.387698,-14.601717,-16.027788,-17.590738,-18.960330,-17.466587,-13.396502,-11.239160,-11.033072,...,-13.988429,-14.628994,-15.409939,-17.380861,-18.928078,-16.615593,-14.877395,-12.847685,-7.911921,-5.437556
41471,1.210966,1.471138,-2.622214,-2.633948,-2.900963,-2.985539,-3.919419,-3.149667,-6.167468,-15.606489,...,-23.452820,-15.567695,-6.553149,-2.219030,-2.544971,-3.233550,-4.279190,-4.625721,-6.232091,-9.236467
30840,-21.946554,-15.952872,-10.554239,-9.379465,-11.117817,-11.506151,-13.084169,-16.424755,-19.063097,-19.811750,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
35672,0.822653,-2.186076,-8.977334,-9.307989,-12.646804,-15.249264,-15.977889,-16.148985,-16.183445,-11.389741,...,-10.923646,-10.501074,-9.541777,-9.952765,-8.756118,-9.389297,-8.264810,-7.081031,-5.752587,-3.924124


In [26]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

lb = LabelEncoder()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)
y_val = lb.transform(y_val)

y_train =  to_categorical(y_train, num_classes=12)
y_test =  to_categorical(y_test, num_classes=12)
y_val =  to_categorical(y_val, num_classes=12)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [27]:
# 세번째 차원을 추가
# -> CNN은 3차원 배열을 입력을 받는다.
x_traincnn =np.expand_dims(X_train, axis=2)
x_testcnn= np.expand_dims(X_test, axis=2)
x_valcnn= np.expand_dims(X_val, axis=2)

In [28]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights = True)
model_checkpoint = ModelCheckpoint('best_model_.h5', save_best_only=True)

In [29]:
# 간단한 RNN 모델 정의 (Bidirectional LSTM 사용, 4층, 12개의 출력 클래스)
model = Sequential()
# model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(12, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(x_traincnn, y_train,
          epochs=100,
          batch_size=32,
          validation_data=(x_valcnn, y_val),
          callbacks = model_checkpoint)

Epoch 1/100
964/964 [==============================] - 101s 88ms/step - loss: 1.7683 - accuracy: 0.3057 - val_loss: 1.6491 - val_accuracy: 0.3411
Epoch 2/100


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


964/964 [==============================] - 84s 87ms/step - loss: 1.5643 - accuracy: 0.3814 - val_loss: 1.5612 - val_accuracy: 0.3931
Epoch 3/100
964/964 [==============================] - 84s 87ms/step - loss: 1.3894 - accuracy: 0.4555 - val_loss: 1.3703 - val_accuracy: 0.4737
Epoch 4/100
964/964 [==============================] - 84s 87ms/step - loss: 1.1862 - accuracy: 0.5458 - val_loss: 1.1124 - val_accuracy: 0.5808
Epoch 5/100
964/964 [==============================] - 85s 89ms/step - loss: 1.0438 - accuracy: 0.6111 - val_loss: 1.0611 - val_accuracy: 0.6029
Epoch 6/100
964/964 [==============================] - 84s 87ms/step - loss: 0.9339 - accuracy: 0.6541 - val_loss: 0.8936 - val_accuracy: 0.6670
Epoch 7/100
964/964 [==============================] - 81s 84ms/step - loss: 0.8842 - accuracy: 0.6774 - val_loss: 0.9087 - val_accuracy: 0.6639
Epoch 8/100
964/964 [==============================] - 84s 87ms/step - loss: 0.8101 - accuracy: 0.7046 - val_loss: 0.8082 - val_accuracy: 0.71

KeyboardInterrupt: ignored

In [17]:
model_name = 'Emotion_Voice_Detection_Model.h5'
save_dir = os.path.join(os.getcwd(), 'saved_models')
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at /content/saved_models/Emotion_Voice_Detection_Model.h5 
